# Using a VAE to expand the training data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import sklearn
from sklearn. model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif

import matplotlib.pyplot as plt # For data viz
import pandas as pd
import numpy as np
import sys

print('System Version:', sys.version)
print('PyTorch version', torch.__version__)
print('CUDA version', torch.version.cuda)
print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)
print('Sklearn version', sklearn.__version__)

In [ ]:
# Confirm device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")
try: 
    name = torch.cuda.get_device_name(0)
    count = torch.cuda.device_count()
    print(f"Device count: {count}")
    print(f"Device name: {name}")
except RuntimeError:
    print('No GPUs detected')

In [ ]:
# Load the data
X = np.load('data/X_TRAIN_RAW.npy')
y = np.load('data/y_TRAIN_RAW.npy')

In [ ]:
# Define the dataset
class COPEDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        cope_data = self.data[index]
        cope_data = (cope_data - np.min(cope_data)) / (np.max(cope_data) - np.min(cope_data))  # normalize

        label = self.target[index]
        volume = torch.tensor(cope_data, dtype=torch.float32).unsqueeze(0)  # (1, 91, 109, 91)
        label = torch.tensor([1.0, 0.0] if label == 0 else [0.0, 1.0], dtype=torch.float32)
        return volume, label

In [ ]:
# Initiate the dataset and data loader
dataset = COPEDataset(X, y)
dataloader = DataLoader(dataset, batch_size=4)

Creating an autoencoder to generate more data

In [ ]:
# Define the model
class CVAE3D(nn.Module):
    def __init__(self, latent_dim=128, label_dim=2, label_embed_dim=16):
        super(CVAE3D, self).__init__()
        self.latent_dim = latent_dim
        self.label_dim = label_dim

        # Embed the label into a dense vector
        self.label_embedding = nn.Linear(label_dim, label_embed_dim)

        # Encoder
        self.encoder_conv = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=3, stride=2, padding=1),  # -> (16, 46, 55, 46)
            nn.BatchNorm3d(16),
            nn.ReLU(),

            nn.Conv3d(16, 32, kernel_size=3, stride=2, padding=1),  # -> (32, 23, 28, 23)
            nn.BatchNorm3d(32),
            nn.ReLU(),

            nn.Conv3d(32, 64, kernel_size=3, stride=2, padding=1),  # -> (64, 12, 14, 12)
            nn.BatchNorm3d(64),
            nn.ReLU(),

            nn.Conv3d(64, 128, kernel_size=3, stride=2, padding=1),  # -> (128, 6, 7, 6)
            nn.BatchNorm3d(128),
            nn.ReLU()
        )

        self.flatten_dim = 128 * 6 * 7 * 6
        self.encoder_fc_mu = nn.Linear(self.flatten_dim + label_embed_dim, latent_dim)
        self.encoder_fc_logvar = nn.Linear(self.flatten_dim + label_embed_dim, latent_dim)

        # Decoder
        self.decoder_fc = nn.Linear(latent_dim + label_embed_dim, self.flatten_dim)

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose3d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(),

            nn.ConvTranspose3d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),

            nn.ConvTranspose3d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),

            nn.ConvTranspose3d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def encode(self, x, labels):
        x = self.encoder_conv(x)
        x = x.view(x.size(0), -1)
        labels = self.label_embedding(labels)
        x = torch.cat([x, labels], dim=1)
        mu = self.encoder_fc_mu(x)
        logvar = self.encoder_fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, labels):
        labels = self.label_embedding(labels)
        z = torch.cat([z, labels], dim=1)
        x = self.decoder_fc(z)
        x = x.view(-1, 128, 6, 7, 6)
        x = self.decoder_conv(x)
        # Crop to original shape
        x = x[:, :, :91, :109, :91]
        return x

    def forward(self, x, labels):
        mu, logvar = self.encode(x, labels)
        z = self.reparameterize(mu, logvar)
        recon = self.decode(z, labels)
        return recon, mu, logvar


In [ ]:
# Define the loss
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')
    kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kld

In [ ]:
# Train the model
model = CVAE3D(latent_dim=128).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(30):
    model.train()
    total_loss = 0
    for batch in dataloader:
        volumes, labels = batch
        volumes, labels = volumes.cuda(), labels.cuda()

        optimizer.zero_grad()
        recon, mu, logvar = model(volumes, labels)
        loss = vae_loss(recon, volumes, mu, logvar)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")


In [ ]:
# Generate synthetic data
n = 200
NEW_X = torch.empty((n*2,91,109,91), device=device)
NEW_y = y
model.eval()

# Generating 40 * 3 new samples (120 samples: 60 of each group)
# Group 0
print('Group 0:')
for i in range(n):
    with torch.no_grad():
        z = torch.randn(1, 128).cuda()

        group_label = torch.tensor([[1.0, 0.0]]).cuda()  # one-hot
        synthetic = model.decode(z, group_label)  # (1, 1, 91, 109, 91)
        synthetic = torch.squeeze(synthetic)
    
        NEW_X[i] = synthetic #, synthetic, axis=0)
        NEW_y = np.append(NEW_y, [0], axis=0)
        print(f'Generated sample {i+1}/200')

# Group 1
print('Group 1:')
for i in range(n):
    with torch.no_grad():
        z = torch.randn(1, 128).cuda()

        group_label = torch.tensor([[0.0, 1.0]]).cuda()  # one-hot
        synthetic = model.decode(z, group_label)  # (1, 1, 91, 109, 91)
        synthetic = torch.squeeze(synthetic)
    
    NEW_X[i+n] = synthetic #NEW_X = np.append(NEW_X, synthetic, axis=0)
    NEW_y = np.append(NEW_y, [1], axis=0)
    print(f'Generated sample {i+1}/200')

Syn_X = NEW_X.cpu().numpy()
NEW_X = np.append(X, Syn_X, axis=0)
        

In [ ]:
# Confirm additions
print(NEW_X.shape)
print(NEW_y.shape)

In [ ]:
# Save the new data
np.save('data/X_SYN.npy', NEW_X)
np.save('data/y_SYN.npy', NEW_y)